In [1]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import re
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem
from rdkit.Chem import Fragments
#import selfies as sf

# 加载保存的模型和分词器路径
#model_path = "./Llama3_8b_QLoRA/checkpoint-535"
#model_path = "./Llama3_8b_mol_ADEMT1/checkpoint-891"
model_path = "/openbayes/input/input2"
model_path1= "./model/llama3_8b_cyp5_epoch_3/"
# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_path1)

# 设置模型为评估模式
model.eval()

# 检查 CUDA 的可用性
if torch.cuda.is_available():
    device = torch.device("cuda")
    model.to(device)
    print("Using CUDA:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

# 定义生成用户提示的函数


### generate prompt based on template ###
prompt_template = {
    "prompt": (
        "Please predict the inhibitory activity for all of the following cytochrome P450 enzymes: CYP1A2, CYP2C9, CYP2C19, CYP2D6, and CYP3A4, based on the provided SMILES and Functional Groups.\n"
        "Assess the activity as either 'active' or 'inactive' for each enzyme. Ensure to provide one prediction for each enzyme and for all 5 enzymes.\n\n"
        "### SMILES:\n{Smiles}\n\n### Functional Groups:\n{functional_groups}\n\n### RESPONSE (for all 5 enzymes):\n"
    ),
    "response_split": "### RESPONSE:"
}

def generate_prompt(instruction,prompt_template=prompt_template):
    # 提取功能团信息
    functional_groups = extract_functional_groups(instruction)
    
    # 格式化功能团信息为字符串
    functional_group_str = ""
    for group, present in functional_groups.items():
        if present:  # 只包括存在的功能团
            functional_group_str += f"- {group}: present\n"
    
    # 使用 SMILES 和功能团生成基础 prompt
    res = prompt_template["prompt"].format(Smiles=instruction, functional_groups=functional_group_str)
    
    return res


def generate_molecular_info(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    molecule = Chem.AddHs(molecule)
    
    # 基本化学信息
    formula = rdMolDescriptors.CalcMolFormula(molecule)
    molecular_weight = Descriptors.ExactMolWt(molecule)
    
    # 理化性质
    logp = Descriptors.MolLogP(molecule)
    tpsa = Descriptors.TPSA(molecule)

    # 分子指纹
    #fingerprint = AllChem.GetMorganFingerprintAsBitVect(molecule, radius=2, nBits=512)
    
    return {
        "formula": formula,
        "molecular_weight": molecular_weight,
        "logp": logp,
        "tpsa": tpsa,
    }

def extract_functional_groups(smiles):
    mol = Chem.MolFromSmiles(smiles)
    # 仅提取与 CYP1A2 代谢相关的功能团
    functional_groups = {
        'Carboxylic Acids': Fragments.fr_COO(mol),              # 羧酸类
        'Esters': Fragments.fr_ester(mol),                      # 酯类
        'Hydroxy Groups': len(mol.GetSubstructMatches(Chem.MolFromSmarts('[OX2H]'))),  # 羟基
        'Amines': Fragments.fr_NH2(mol),                        # 胺类
        'Ethers': Fragments.fr_ether(mol),                      # 醚类
        'Phenols': Fragments.fr_phenol(mol),                    # 酚类
        'Benzene Rings': Fragments.fr_benzene(mol),             # 苯环
        'Aromatic Amines': Fragments.fr_Ar_N(mol),              # 芳香胺类
        'Carbonyl Groups': Fragments.fr_C_O(mol),               # 羰基类
        'Ketones': Fragments.fr_ketone(mol),                    # 酮类
        'Amides': Fragments.fr_amide(mol),                      # 酰胺类
        'Nitriles': Fragments.fr_nitrile(mol)                   # 腈类
    }
    return functional_groups
def generate_text(smiles, prompt_template):
    prompt = generate_prompt(smiles,prompt_template)

    return prompt

def generate_selfies(data_point):
    molecule = Chem.MolFromSmiles(data_point)
    selfies_string = sf.encoder(Chem.MolToSmiles(molecule))
    return selfies_string

# 加载数据集
dataset = pd.read_csv("new_test.csv")
dataset = dataset.head(1000)

# 定义要预测的酶
enzymes = ['1a2', '2c9', '2c19', '2d6', '3a4']

# 定义酶与生成响应中对应的标签
enzyme_labels = {
    '1a2': 'CYP1A2',
    '2c9': 'CYP2C9',
    '2c19': 'CYP2C19',
    '2d6': 'CYP2D6',
    '3a4': 'CYP3A4'
}

# 初始化变量
total_non_missing_count = 0
total_correct_count = 0

# 新增：初始化每种酶的正确预测和非缺失统计
enzyme_correct_counts = {enzyme: 0 for enzyme in enzymes}
enzyme_non_missing_counts = {enzyme: 0 for enzyme in enzymes}

for index, data_point in dataset.iterrows():
    # 生成 selfies 和功能团信息
    user_prompt = generate_prompt(data_point["Smiles"], prompt_template)
    inputs = tokenizer(user_prompt, return_tensors="pt").to(device)

    # 使用模型生成响应
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=80,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            num_return_sequences=1,
            repetition_penalty=1.2  # 避免重复生成
        )
    
    # 解码生成的响应
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 提取生成的标签，忽略大小写
    generated_labels = {}
    for enzyme in enzymes:
        label = enzyme_labels[enzyme]
        match = re.search(f"{label}: (active|inactive)", generated_text, re.IGNORECASE)
        if match:
            generated_labels[enzyme] = 1.0 if match.group(1).lower() == 'active' else 0.0
    
    # 检查是否生成了5个酶的标签
    if len(generated_labels) == 5:
        correct_count = 0
        non_missing_count = 0
        for enzyme in enzymes:
            if not pd.isna(data_point[enzyme]):
                non_missing_count += 1
                enzyme_non_missing_counts[enzyme] += 1  # 统计每个酶的非缺失样本数
                
                if enzyme in generated_labels and data_point[enzyme] == generated_labels[enzyme]:
                    correct_count += 1
                    enzyme_correct_counts[enzyme] += 1  # 统计每个酶的正确预测次数
        
        total_non_missing_count += non_missing_count
        total_correct_count += correct_count
    else:
        print("Model did not generate labels for all 5 enzymes.")

# 计算并打印总体ACC和每种酶的ACC
total_accuracy = total_correct_count / total_non_missing_count if total_non_missing_count > 0 else 0
print(f"Total Accuracy: {total_accuracy:.4f}")

for enzyme in enzymes:
    enzyme_accuracy = enzyme_correct_counts[enzyme] / enzyme_non_missing_counts[enzyme] if enzyme_non_missing_counts[enzyme] > 0 else 0
    print(f"{enzyme} Accuracy: {enzyme_accuracy:.4f}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated text: Please predict the inhibitory activity for all of the following cytochrome P450 enzymes: CYP1A2, CYP2C9, CYP2C19, CYP2D6, and CYP3A4, based on the provided SMILES and Functional Groups.
Assess the activity as either 'active' or 'inactive' for each enzyme. Ensure to provide one prediction for each enzyme and for all 5 enzymes.

### SMILES:
COc1cc2c(cc1OC)C(=O)N(C)CC2

### Functional Groups:
- Ethers: present
- Benzene Rings: present
- Carbonyl Groups: present
- Amides: present


### RESPONSE (for all 5 enzymes):
CYP2C19: inactive
CYP2D6: inactive
CYP3A4: inactive
CYP2C9: inactive
CYP2C11: inactive
CYP2D7: inactive
CYP2D8: inactive
CYP2D9: inactive
CYP2D10: inactive
CYP2C19: inactive

Model did not generate labels for all 5 enzymes.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated text: Please predict the inhibitory activity for all of the following cytochrome P450 enzymes: CYP1A2, CYP2C9, CYP2C19, CYP2D6, and CYP3A4, based on the provided SMILES and Functional Groups.
Assess the activity as either 'active' or 'inactive' for each enzyme. Ensure to provide one prediction for each enzyme and for all 5 enzymes.

### SMILES:
COc1ccc(NC(=O)c2cc(C(C)C)on2)cc1OC

### Functional Groups:
- Ethers: present
- Benzene Rings: present
- Aromatic Amines: present
- Carbonyl Groups: present
- Amides: present


### RESPONSE (for all 5 enzymes):
CYP3A4: inactive
CYP2C19: active
CYP2D6: inactive
CYP3A4: inactive
CYP2C9: inactive
CYP2D6: inactive
CYP3A4: inactive
CYP2C19: inactive
CYP2D6: inactive
CYP3A4: inactive

Model did not generate labels for all 5 enzymes.


...

Generated text: Please predict the inhibitory activity for all of the following cytochrome P450 enzymes: CYP1A2, CYP2C9, CYP2C19, CYP2D6, and CYP3A4, based on the provided SMILES and Functional Groups.
Assess the activity as either 'active' or 'inactive' for each enzyme. Ensure to provide one prediction for each enzyme and for all 5 enzymes.

### SMILES:
CC(C)NC(=O)N1CCC2(CCCN(S(=O)(=O)c3ccccc3)C2)C1

### Functional Groups:
- Benzene Rings: present
- Carbonyl Groups: present
- Amides: present


### RESPONSE (for all 5 enzymes):
CYP1A2: inactive
CYP2C9: inactive
CYP2C19: inactive
CYP2D6: active
CYP3A4: inactive
CYP2C9: inactive
CYP2D6: inactive
CYP3A4: inactive
CYP2C19: inactive
CYP2D6: inactive

Correct labels count: 0, Non-missing features count: 1


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated text: Please predict the inhibitory activity for all of the following cytochrome P450 enzymes: CYP1A2, CYP2C9, CYP2C19, CYP2D6, and CYP3A4, based on the provided SMILES and Functional Groups.
Assess the activity as either 'active' or 'inactive' for each enzyme. Ensure to provide one prediction for each enzyme and for all 5 enzymes.

### SMILES:
O=C(c1ccco1)N1CCC2(CCN(Cc3cc(C(F)(F)F)cc(C(F)(F)F)c3)CC2)CC1

### Functional Groups:
- Benzene Rings: present
- Carbonyl Groups: present
- Amides: present


### RESPONSE (for all 5 enzymes):
CYP2D6: inactive
CYP3A4: active
CYP1A2: inactive
CYP2C9: inactive
CYP2C19: inactive
CYP2D6: inactive
CYP3A4: inactive
CYP1A2: inactive
CYP2C19: inactive
CYP2D6: inactive

Correct labels count: 0, Non-missing features count: 1


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated text: Please predict the inhibitory activity for all of the following cytochrome P450 enzymes: CYP1A2, CYP2C9, CYP2C19, CYP2D6, and CYP3A4, based on the provided SMILES and Functional Groups.
Assess the activity as either 'active' or 'inactive' for each enzyme. Ensure to provide one prediction for each enzyme and for all 5 enzymes.

### SMILES:
CC(C)C1=CC2=C(C=C1)C3(CCCC(C3CC2)(C)CN)C

### Functional Groups:
- Amines: present
- Benzene Rings: present


### RESPONSE (for all 5 enzymes):
CYP1A2: active
CYP2C9: inactive
CYP2C19: active
CYP2D6: active
CYP3A4: active
CYP2C19: active
CYP2D6: active
CYP3A4: active
CYP2C9: active
CYP2C19: active

Correct labels count: 1, Non-missing features count: 1
Generated text: Please predict the inhibitory activity for all of the following cytochrome P450 enzymes: CYP1A2, CYP2C9, CYP2C19, CYP2D6, and CYP3A4, based on the provided SMILES and Functional Groups.
Assess the activity as either 'active' or 'inactive' for each enzyme. Ensure to provid